In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc, html, Input, Output
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
import base64
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import socket

# Import your CRUD class
from CRUD import CRUD

# JupyterDash configuration
JupyterDash.infer_jupyter_proxy_config()

# MongoDB connection setup
username = "aacuser"
password = "nathan"
host = "localhost"
port = 27017
database = "aac"
collection = "animals"

# Initialize CRUD object
db = CRUD(username, password)

# Read all documents from MongoDB
df = pd.DataFrame.from_records(db.read({}))

# Drop the '_id' column which is not needed for display
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Print column names to verify
print(df.columns)

# Initialize the app
app = JupyterDash(__name__)

# Encode the logo
with open("Grazioso Salvare Logo.png", "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read())

# Define the app layout
app.layout = html.Div([
    # Hidden div for callbacks
    html.Div(id='hidden-div', style={'display':'none'}),
    
    # Logo and creator name
    html.Div([
        html.A(
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                     style={'height':'80px'}),
            href='https://www.snhu.edu',
            target="_blank"
        ),
        html.Div("Created by Nathan Sakhichand", style={'textAlign': 'center', 'fontSize': 14})
    ]),
    
    # Title
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    
    # Filter options
    html.Div(className='buttonRow', 
             style={'display' : 'flex'},
             children=[
                 dcc.RadioItems(
                     id='filter-type',
                     options=[
                         {'label': 'Water Rescue', 'value': 'water'},
                         {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                         {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                         {'label': 'Reset', 'value': 'reset'}
                     ],
                     value='reset',
                     labelStyle={'display': 'inline-block', 'margin-right': '10px'}
                 )
             ]),
    
    # Interactive Data Table
    dcc.Loading(
        id="loading-1",
        type="default",
        children=dash_table.DataTable(
            id='datatable-id',
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
            data=df.to_dict('records'),
            editable=False,
            filter_action="native",
            sort_action="native",
            sort_mode="multi",
            column_selectable=False,
            row_selectable="single",
            row_deletable=False,
            selected_columns=[],
            selected_rows=[],
            page_action="native",
            page_current=0,
            page_size=10
        )
    ),
    
    # Dashboard elements
    html.Div([
        # Pie chart
        html.Div(id='graph-id', style={'width': '49%', 'display': 'inline-block'}),
        # Map
        html.Div(id='map-id', style={'width': '49%', 'display': 'inline-block'})
    ]),
    
    html.Br(),
    html.Hr()
])

# Callback to update the data table
@app.callback(Output('datatable-id','data'),
              [Input('filter-type','value')])
def update_table(filter_type):
    if filter_type == "water":
        criteria = {
            "animal_type": "Dog",
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
            "breed": {
                "$in": [
                    "Labrador Retriever Mix",
                    "Chesapeake Bay Retriever",
                    "Newfoundland"
                ]
            }
        }
    elif filter_type == "mountain":
        criteria = {
            "animal_type": "Dog",
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":26, "$lte": 156},
            "breed": {
                "$in": [
                    "German Shepherd",
                    "Alaskan Malamute",
                    "Old English Sheepdog",
                    "Siberian Husky",
                    "Rottweiler"
                ]
            }
        }
    elif filter_type == "disaster":
        criteria = {
            "animal_type": "Dog",
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300},
            "breed": {
                "$in": [
                    "Doberman Pinscher",
                    "German Shepherd",
                    "Golden Retriever",
                    "Bloodhound",
                    "Rottweiler"
                ]
            }
        }
    else:  # reset
        criteria = {}

    records = db.read(criteria)
    df_filtered = pd.DataFrame.from_records(records)
    if not df_filtered.empty and '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)
    
    return df_filtered.to_dict('records')

# Callback to update the pie chart
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data')]
)
def update_graphs(rows):
    if rows is None or len(rows) == 0:
        print("No data available for pie chart")
        return []
    dff = pd.DataFrame(rows)
    if 'breed' not in dff.columns:
        print("Breed column not found in data")
        return []

    fig = px.pie(dff, names='breed', title='Preferred Breeds Distribution')
    return [dcc.Graph(figure=fig)]

# Callback to update the map with updated field names and structure
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(rows, selected_rows):
    if rows is None or len(rows) == 0:
        # Return an empty map centered at Austin TX, with just tile layer
        return [
            dl.Map(center=[30.2672, -97.7431], zoom=10, style={'width': '100%', 'height': '50vh'}, children=[
                dl.TileLayer()
            ])
        ]
    
    dff = pd.DataFrame(rows)
    lat_col = 'location_lat'
    lon_col = 'location_long'
    
    if lat_col not in dff.columns or lon_col not in dff.columns:
        # Return empty base map if missing columns
        return [dl.Map(center=[30.2672, -97.7431], zoom=10, style={'width': '100%', 'height': '50vh'}, children=[dl.TileLayer()])]

    # Default center is first row's location
    center = [dff[lat_col].iloc[0], dff[lon_col].iloc[0]] if not dff.empty else [30.2672, -97.7431]
    
    # Create markers for each row
    markers = []
    for _, row in dff.iterrows():
        if pd.notnull(row[lat_col]) and pd.notnull(row[lon_col]):
            marker = dl.Marker(position=[row[lat_col], row[lon_col]], 
                               title=f"{row['name']}, {row['breed']}")
            markers.append(marker)
        else:
            print(f"Missing location data for {row['name']}")

    # Highlight selected row if any
    if selected_rows and len(selected_rows) > 0:
        selected_row = dff.iloc[selected_rows[0]]
        if pd.notnull(selected_row[lat_col]) and pd.notnull(selected_row[lon_col]):
            # Use default marker for selected row, no custom icon
            markers.append(dl.Marker(position=[selected_row[lat_col], selected_row[lon_col]], 
                                     title=f"Selected: {selected_row['name']}, {selected_row['breed']}"))
        else:
            print("Selected row has no valid location data")

    # Map setup
    map_children = [
        dl.Map(center=center, zoom=10, style={'width': "100%", 'height': "50vh", 'margin': "auto", 'display': 'block'}, children=[
            dl.TileLayer(),
            *markers
        ])
    ]

    return map_children

# Function to find a free port
def find_free_port():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(('', 0))
    _, port = sock.getsockname()
    sock.close()
    return port

# Run the app with a free port
if __name__ == '__main__':
    port = find_free_port()
    app.run_server(mode='jupyterlab', port=port)



/tmp/ipykernel_676/226658421.py:8: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks', 're_num',
       'Location_long'],
      dtype='object')
No data available for pie chart
Missing location data for Max
Missing location data for Max
Missing location data for Max
Missing location data for Max
Missing location data for Max
Missing location data for Max
